In [10]:
# Required libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import Adam
import tensorflow as tf
from keras.utils import register_keras_serializable

In [3]:

dataset_path = 'dataset.csv'
dataset = pd.read_csv(dataset_path)

In [4]:
y = dataset['Churn Label'].values
X = dataset.drop(['Churn Label', 'Customer ID'], axis=1)

In [5]:
categorical_cols = X.select_dtypes(include=['object']).columns
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [7]:
class ButterflyOptimizationAlgorithm:
    def __init__(self, fitness_function, X, y, pop_size=20, max_iter=100, a=0.01, c=0.1):
        self.fitness_function = fitness_function
        self.X = X
        self.y = y
        self.pop_size = pop_size
        self.max_iter = max_iter
        self.a = a
        self.c = c
        self.population = None
        self.best_solution = None
        self.best_fitness = -np.inf

    def initialize_population(self, dim):
        self.population = np.random.rand(self.pop_size, dim)

    def optimize(self, dim):
        self.initialize_population(dim)
        for iter in range(self.max_iter):
            for i in range(self.pop_size):
                new_solution = self.population[i] + np.random.normal(0, 1, dim) * self.a
                new_fitness = self.fitness_function(new_solution, self.X, self.y)  # Pass X and y to the fitness function
                if new_fitness > self.best_fitness:
                    self.best_fitness = new_fitness
                    self.best_solution = new_solution
            self.population += np.random.normal(0, 1, (self.pop_size, dim)) * self.c
        return self.best_solution

In [8]:
def fitness_function(solution, X, y):
    selected_features = np.where(solution > 0.5, 1, 0)
    if np.sum(selected_features) == 0:
        return 0
    X_selected = X[:, selected_features == 1]
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
    model = Sequential()
    model.add(LSTM(64, input_shape=(X_train.shape[1], 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=40, batch_size=32, verbose=0)
    accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
    return accuracy

In [11]:

@register_keras_serializable()
def swish(x):
    return x * tf.keras.activations.sigmoid(x)


In [12]:
def build_model(X, y):
    boa = ButterflyOptimizationAlgorithm(fitness_function=fitness_function, X=X, y=y)
    best_solution = boa.optimize(X.shape[1])
    selected_features = np.where(best_solution > 0.5, 1, 0)
    X_selected = X[:, selected_features == 1]
    
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
    
    model = Sequential()
    model.add(LSTM(64, input_shape=(X_train.shape[1], 1)))
    model.add(Dense(1, activation=swish))  # Swish activation
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=60, batch_size=32, verbose=1)

    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: ", accuracy)
    return model

In [26]:

# Build and train the model
model = build_model(X_scaled, y)
model_save_path = 'trained_churn_model_v2.keras'
model.save(model_save_path)

c:\Users\syedf\anaconda3\envs\sapy\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
